##Author: Sanjeet Maisnam

# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

##Start the Spark Session

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('churn').getOrCreate()


##Exploratory Data Analysis

In [7]:
data = spark.read.csv('FileStore/tables/customer_churn.csv', inferSchema = True, header = True)

In [8]:
data.columns

Out[3]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
data.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [10]:
data.describe()

Out[5]: DataFrame[summary: string, Names: string, Age: string, Total_Purchase: string, Account_Manager: string, Years: string, Num_Sites: string, Location: string, Company: string, Churn: string]

In [11]:
data.corr(data.columns[4],data.columns[9])

Out[6]: 0.21432858017242842

In [12]:
data.select('Churn').distinct().count()

Out[7]: 2

##Feature Selection

In [14]:
my_cols = data.select(['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Churn'])

In [15]:
final_data = my_cols.na.drop()

In [16]:
final_data.show(4)

+----+--------------+---------------+-----+---------+-----+
 Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+----+--------------+---------------+-----+---------+-----+
42.0| 11066.8| 0| 7.22| 8.0| 1|
41.0| 11916.22| 0| 6.5| 11.0| 1|
38.0| 12884.75| 0| 6.67| 12.0| 1|
42.0| 8010.76| 0| 6.71| 10.0| 1|
+----+--------------+---------------+-----+---------+-----+
only showing top 4 rows

In [17]:
final_data.columns[:-1]

Out[11]: ['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites']

In [18]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = final_data.columns[:-1], outputCol = 'features')

##Build ML Pipeline and Train the model

In [20]:
from pyspark.ml.classification import LogisticRegression

In [21]:
from pyspark.ml import Pipeline

In [22]:
model = LogisticRegression(featuresCol = 'features', labelCol = 'Churn')

In [23]:
pipeline = Pipeline(stages=[assembler, model])

In [24]:
train_set, new_customers = final_data.randomSplit([0.65, 0.35])

In [25]:
new_customers.show()

+----+--------------+---------------+-----+---------+-----+
 Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+----+--------------+---------------+-----+---------+-----+
27.0| 8628.8| 1| 5.3| 7.0| 0|
28.0| 9090.43| 1| 5.74| 10.0| 0|
28.0| 11245.38| 0| 6.72| 8.0| 0|
29.0| 8688.17| 1| 5.7| 9.0| 1|
29.0| 10203.18| 1| 5.82| 8.0| 0|
30.0| 8677.28| 1| 7.31| 7.0| 0|
30.0| 11575.37| 1| 5.22| 8.0| 1|
30.0| 13473.35| 0| 3.84| 10.0| 0|
31.0| 5387.75| 0| 6.83| 9.0| 0|
31.0| 10058.87| 1| 6.54| 7.0| 0|
31.0| 10182.6| 1| 3.77| 8.0| 0|
32.0| 6367.22| 1| 2.84| 10.0| 0|
32.0| 8011.38| 0| 5.35| 10.0| 0|
32.0| 8617.98| 1| 6.2| 10.0| 1|
32.0| 9885.12| 1| 6.92| 9.0| 1|
32.0| 12142.99| 0| 5.01| 7.0| 0|
32.0| 12403.6| 0| 4.99| 7.0| 0|
32.0| 12479.72| 0| 4.67| 8.0| 0|
32.0| 13630.93| 0| 4.38| 10.0| 0|
33.0| 4711.89| 0| 4.97| 7.0| 0|
+----+--------------+---------------+-----+---------+-----+
only showing top 20 rows

In [26]:

fit_model = pipeline.fit(train_set)

In [27]:
testing = fit_model.transform(new_customers)

In [28]:
testing.head(5)

Out[48]: [Row(Age=27.0, Total_Purchase=8628.8, Account_Manager=1, Years=5.3, Num_Sites=7.0, Churn=0, features=DenseVector([27.0, 8628.8, 1.0, 5.3, 7.0]), rawPrediction=DenseVector([6.3166, -6.3166]), probability=DenseVector([0.9982, 0.0018]), prediction=0.0),
 Row(Age=28.0, Total_Purchase=9090.43, Account_Manager=1, Years=5.74, Num_Sites=10.0, Churn=0, features=DenseVector([28.0, 9090.43, 1.0, 5.74, 10.0]), rawPrediction=DenseVector([2.04, -2.04]), probability=DenseVector([0.8849, 0.1151]), prediction=0.0),
 Row(Age=28.0, Total_Purchase=11245.38, Account_Manager=0, Years=6.72, Num_Sites=8.0, Churn=0, features=DenseVector([28.0, 11245.38, 0.0, 6.72, 8.0]), rawPrediction=DenseVector([4.4932, -4.4932]), probability=DenseVector([0.9889, 0.0111]), prediction=0.0),
 Row(Age=29.0, Total_Purchase=8688.17, Account_Manager=1, Years=5.7, Num_Sites=9.0, Churn=1, features=DenseVector([29.0, 8688.17, 1.0, 5.7, 9.0]), rawPrediction=DenseVector([3.2724, -3.2724]), probability=DenseVector([0.9635, 0.0365]), prediction=0.0),
 Row(Age=29.0, Total_Purchase=10203.18, Account_Manager=1, Years=5.82, Num_Sites=8.0, Churn=0, features=DenseVector([29.0, 10203.18, 1.0, 5.82, 8.0]), rawPrediction=DenseVector([4.4557, -4.4557]), probability=DenseVector([0.9885, 0.0115]), prediction=0.0)]

In [29]:
testing.describe()

Out[49]: DataFrame[summary: string, Age: string, Total_Purchase: string, Account_Manager: string, Years: string, Num_Sites: string, Churn: string, prediction: string]

##Model Evaluation

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [32]:
model_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [33]:
testing.select('Churn','prediction').show()

+-----+----------+
Churn|prediction|
+-----+----------+
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+-----+----------+
only showing top 20 rows

In [34]:
AUC = model_eval.evaluate(testing)
AUC

Out[53]: 0.7764646464646464